In [ ]:


parsed_file = "test_file"
word_occurence_map = {}

with open(parsed_file) as inp:
    prev_line, prev_count = '', 0
    for line in inp:
        curr_line = "\t".join(line.strip().split("\t")[:-1])
        if curr_line == prev_line or (not prev_line):
            prev_count += int(line.strip().split("\t")[-1])
        else:
            print (curr_line)
            x, y, path = prev_line.split('\t')
            key = str(x) + '_' + str(y)
            current = path + ":" + str(prev_count)
            if key in word_occurence_map:
                pastkeys = word_occurence_map[key]
                current =  pastkeys + ',' + current
            word_occurence_map[key] = current
            prev_count = 1
        prev_line = curr_line
    x, y, path = prev_line.split('\t')
    key = str(x) + '_' + str(y)
    current = path + ":" + str(prev_count)
    if key in word_occurence_map:
        pastkeys = word_occurence_map[key]
        current =  pastkeys + ',' + current
    word_occurence_map[key] = current
    


In [ ]:
import sys, shelve, picklw
file = "damru"
p = pickle.load(open(file, 'rb'))
db_file = ".".join(file.split(".")[:-1]) + ".db" 
with shelve.open(db_file, 'c') as db:
    for k in p:
        try:
            p[k] = p[k].decode("utf-8")
        except:
            pass
        try:
            k = k.decode("utf-8")
        except:
            pass

        if k and p[k]:
            db[str(k)] = str(p[k])
print ("db_file", db_file, "done")

In [ ]:
import os,shelve
folders = ["../junk/Files/" + l + "/" for l in os.listdir("../junk/Files/") if l.startswith("security_threshold_")]

passed1 = []
for f in folders:
    if len(list(shelve.open(f + "security_relations_map.db", "r").items())):
        passed1.append(f)

print ("Folders having uncorrupted relation dict {}/{}".format(len(passed1), len(folders)))

passed2 = []
for f in passed1:
    if len(list(shelve.open(f + "security_path_to_id_dict.db", "r").items())) and len(list(shelve.open(f + "security_id_to_path_dict.db", "r").items())):
        passed2.append(f)

print ("Folders having uncorrupted path-to-id dict {}/{}".format(len(passed2), len(folders)))

passed3,passed4, passed5 = [], [], []

for f in passed2:
    s1 = list(shelve.open(f + "security_id_to_word_dict.db", "r").keys())
    s2 = list(shelve.open(f + "security_word_to_id_dict.db", "r").values())
    if len(s1) and len(s2):
        passed3.append(f)
        s1_sort = sorted(s1, key=lambda l:float(l))
        s2_sort = sorted(s2, key=lambda l:float(l))
        if s1_sort[0] == 0 and s2_sort[0] == 0:
            passed4.append(f)
        if s1_sort == s2_sort:
            passed5.append(f)


print ("Folders having uncorrupted word-to-id dict {}/{}".format(len(passed3), len(folders)))
print ("Folders having zero-indexed word-to-id dict {}/{}".format(len(passed4), len(folders)))
print ("Folders having equal word-to-id and id-to-word dicts {}/{}".format(len(passed5), len(folders)))



In [ ]:
print ("Folders having uncorrupted relation dict {}/{}".format(2, 4))

In [ ]:
dataset = open("../files/dataset/dataset.tsv").read().split("\n")
dataset_t = [l for l in dataset if l.split("\t")[-1]!="none"]
dataset_f = [l for l in dataset if l.split("\t")[-1]=="none"]
import random
dataset_t_mini = random.choices(dataset_t, k=int(0.1 * len(dataset_t)))
dataset_f_mini = random.choices(dataset_f, k=10000-int(0.1 * len(dataset_t)))
dataset_mini = dataset_t_mini + dataset_f_mini
random.shuffle(dataset_mini)
open("../files/dataset/dataset_mini.tsv","w+").write("\n".join(dataset_mini))
open("../files/dataset/train_mini.tsv","w+").write("\n".join(dataset_mini[:int(0.9*len(dataset_mini))]))
open("../files/dataset/test_mini.tsv","w+").write("\n".join(dataset_mini[int(0.9*len(dataset_mini)):]))

In [ ]:
from pronto import Ontology
import re

def camel_case_split(identifier):
    matches = re.finditer('.+?(?:(?<=[a-z])(?=[A-Z])|(?<=[A-Z])(?=[A-Z][a-z])|$)', identifier)
    return " ".join([m.group(0).lower() for m in matches])

ont = Ontology("/Users/vivek/SIREN-Research/securityontology.owl")
ontology = [camel_case_split(a.id.split("#")[-1])+"\tcontrol\thypernym" for a in ont.terms()]
import itertools

ontology = list(itertools.chain.from_iterable(itertools.repeat(x, 3) for x in ontology))

In [ ]:
lines = [l for l in open("temp_policy","r").read().split("\n") if "policy" in l.split("\t")[0]]
final = []
for l in lines:
    final.append(l.split("\t")[0] + "\tpolicy\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tpolicy\thypernym")
    final.append(l.split("\t")[0] + "\tintangible asset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tintangible asset\thypernym")
    final.append(l.split("\t")[0] + "\tasset\thypernym")
    final.append(" ".join(l.split("\t")[0].split()[:-1]) + "\tasset\thypernym")
    

In [ ]:
open("../junk/policy","w+").write("\n".join(final))

In [8]:
import shelve
s1 = shelve.open("../junk/Files/security_threshold_7_10/security_word_to_id_dict.db")
s2 = shelve.open("../junk/Files/security_threshold_7_10/security_id_to_word_dict.db")

In [10]:
# s2_rev = {s2[k]: k for k in s2}
missed = []
for word in s1:
    s1_id = s1[word]
    try:
        s2_word = s2[s1_id]
        if word == s2_word:
            continue
        else:
            print ("Word mismatch", word, s1_id, s2_id)

    except:
        try:
            s2_id = s2_rev[word]
            if s2_id != s1_id:
                print ("ID mismatch", word, s1_id, s2_id)

        except:            
            print (word, "not in vocab")
            missed.append(word)
            continue
    

In [9]:
s2_rev = {s2[k]: k for k in s2}
missed = []
for word in s2_rev:
    s2_rev_id = s2_rev[word]
    try:
        s1_id = s1[word]
        if s1_id == s2_rev_id:
            continue
        else:
            print ("ID mismatch", word, s1_id, s2_rev_id)

    except:
        print (word, "not in vocab")
        missed.append(word)
        continue
    

ID mismatch petru 13624688 13624424
ID mismatch april july 3673057 3673056
ID mismatch a shark tank 4064655 4064654
ID mismatch comorbid diseases 6064645 6064644
ID mismatch psg 14091087 14091086
ID mismatch the bias variance tradeoff 17030198 17030197
ID mismatch a solid start 4208491 4208489
ID mismatch artists spaces 3959143 3959142
ID mismatch re 24110341 14297194
ID mismatch a terror attack 4409068 4409067
ID mismatch governor reagan 8409048 8409047
ID mismatch the radio direction finder 20923661 20923660
ID mismatch third floor level 22523653 22523652
ID mismatch the times critic 21946174 21946172
ID mismatch malcolm x 11222985 11222979
ID mismatch panzer groups 13422971 13422970
ID mismatch a predella 3656836 3656835
ID mismatch bil hole 5056829 5048809
ID mismatch butler s work 5456827 5456826
ID mismatch nathan der weise 12245494 12245493
ID mismatch summer isles 16045475 16045474
ID mismatch naval warfare 12283851 12283850
ID mismatch his own late night show 9125120 9125119
I

ID mismatch scots magazine 14880757 14880756
ID mismatch a common council 1168929 1168928
ID mismatch brian markinson 5340570 5340568
ID mismatch civil service reform 5882818 5882816
ID mismatch the golden bowl 18543752 18543750
ID mismatch a cold light 1151155 1151154
ID mismatch chausseestrasse 5751130 5751129
ID mismatch ej rcito revolucionario del pueblo 7151123 7151122
ID mismatch sratsimir 15766282 15766281
ID mismatch fasttimes 7626645 7626644
ID mismatch a higher truth 1826674 1826673
ID mismatch catholicism 5626657 5626656
ID mismatch j k hume 10211066 10211065
ID mismatch kawsampi 10411065 10411064
ID mismatch a negative peace 2829411 2829410
ID mismatch amphibious operations employment 2543325 2543324
ID mismatch an a to f scale 2743324 2743323
ID mismatch the raging storm 20927012 20927011
ID mismatch de spectaculis 6601053 6601052
ID mismatch gender 8201051 8201046
ID mismatch water chenla 23685048 23685047
ID mismatch the bicycle kick 17032094 17032093
ID mismatch the cas

ID mismatch a 0 to 60 mph time 730025 730024
ID mismatch a blues 930024 930023
ID mismatch akif 1938754 1938753
ID mismatch furcifer 8101984 8101983
ID mismatch any set backs 3463222 3463221
ID mismatch a revenge 3863223 3863221
ID mismatch io 12854249 9741179
ID mismatch england s prince charles 7263203 7263202
ID mismatch r union island 14487094 14487093
ID mismatch the great day 18578462 18578461
ID mismatch duncan 6902572 6902571
ID mismatch points 24128480 13755971
ID mismatch prey species 13955965 13955964
ID mismatch sharpies 15155959 15155958
ID mismatch spring br 15755956 15755955
ID mismatch the buddy system 17155950 17155948
ID mismatch nunn 12778488 12778487
ID mismatch naukma 12279860 12279859
ID mismatch an encrypted subscription programming signal 2855404 2855403
ID mismatch port number 13823240 13823238
ID mismatch living machine 10947141 10947139
ID mismatch ripe ncc 14547123 14547121
ID mismatch sz l 16147114 16146475
ID mismatch the goldwasser micali cryptosystem 185

ID mismatch lord faringdon 11030156 11030155
ID mismatch the times building 21946101 21946100
ID mismatch the agence congolaise de presse 16676757 16676756
ID mismatch auto implemented properties 4625224 4625223
ID mismatch new 12382712 12382394
ID mismatch m an 11521539 11239261
ID mismatch johnny ringo mcdonagh 10245432 10245431
ID mismatch prologues 14045413 14045412
ID mismatch rotsi 14645410 14645409
ID mismatch telephone 16245405 16245397
ID mismatch the uss okinawa 22220758 22220757
ID mismatch wasted votes 23681625 23681622
ID mismatch winona ryder 23881624 23881623
ID mismatch x n 24128508 23980764
ID mismatch katsinam 10407074 10407073
ID mismatch processing resources 14007056 14007055
ID mismatch hurricane katrina 9384268 9384267
ID mismatch japanese gaming s top ten hottest girls 10136818 10136817
ID mismatch a true ruler 4522916 4522915
ID mismatch two tyrants 23144640 23144639
ID mismatch the london free press 19565078 19565077
ID mismatch the western dominated world bank

ID mismatch achars 1100583 1100582
ID mismatch good lyrics 8379697 8379696
ID mismatch c f 24126025 5700550
ID mismatch 3 feet 468349 468348
ID mismatch murray brown 12143711 12143710
ID mismatch magna cum laude 11166239 11166238
ID mismatch skylines 15366212 15366211
ID mismatch n low saxon 12505352 12505351
ID mismatch publications 14105351 14105349
ID mismatch russian 14705483 14705277
ID mismatch the high esteem 18705321 18705320
ID mismatch a classic blend 1128922 1128921
ID mismatch a feature film adaptation 1528920 1528919
ID mismatch the ultimate edition 22118479 22118478
ID mismatch centauri 24126023 24121044
ID mismatch hab kina 8546278 8546277
ID mismatch dr king s assassination 6860199 6860198
ID mismatch line mode browser 10900921 10900920
ID mismatch every a 7451796 7451795
ID mismatch pterocarpus 14100906 14100904
ID mismatch seaheath 14900901 14900900
ID mismatch 11 km 69277 69272
ID mismatch the best oil 17017777 17017776
ID mismatch 25 35 years 338393 338392
ID mismat

...

ID mismatch the third largest category 21907712 21907711
ID mismatch cabot s 5477541 5477540
ID mismatch illegal acts 9477524 9477522
ID mismatch woodward 23913492 23913489
ID mismatch ceromonies 5678119 5678118
ID mismatch primavera 13966530 13966528
ID mismatch an academy 2594210 2594209
ID mismatch the most sophisticated professionalized and successful doping program 19972289 19972288
ID mismatch pgpsem 13629543 13629542
ID mismatch micca 11629553 11629552
ID mismatch blood 24152889 5119639
ID mismatch cavalry 5633560 5633554
ID mismatch the initial resistance 18896181 18896180
ID mismatch a campbell 1044147 1044146
ID mismatch the five speed gearbox 18296184 18296183
ID mismatch the story set up 21710519 21710518
ID mismatch a black and white monitor 919029 919028
ID mismatch a modest home 2491902 2491901
ID mismatch the beatles influence 16987576 16987575
ID mismatch much one 12088970 12088969
ID mismatch american psycho 2436319 2436318
ID mismatch the 2015 16 budget year 16493209

ID mismatch the blood sword 17070825 17070824
ID mismatch the last vestiges 19432456 19432455
ID mismatch monsters inc 11832497 11832496
ID mismatch a great artist 1759381 1759378
ID mismatch deep green 6523197 6523196
ID mismatch green consumerism 8465439 8465438
ID mismatch the tinman 21948443 21948442
ID mismatch an anarchism 2676031 2676030
ID mismatch british professional bodies 5370967 5370966
ID mismatch first world standards 7770955 7770954
ID mismatch the exterior 18047735 18047732
ID mismatch the icons 18809374 18809373
ID mismatch carletto 5582108 5582107
ID mismatch a voucher program 4706987 4706986
ID mismatch concentric ridges 6106978 6106977
ID mismatch multiple criteria decision analysis 12115039 12115037
ID mismatch athi 4420899 4419610
ID mismatch the highlight reel 18715006 18715004
ID mismatch bayad 4918126 4918125
ID mismatch densovirina 6573703 6573702
ID mismatch bahriye 4832039 4832038
ID mismatch drag n 6832178 6832177
ID mismatch zelbinion 24075125 24075124
ID

ID mismatch 1 width 253315 53023
ID mismatch scepanotrocha 14848462 14848461
ID mismatch one s lord 12954102 12954101
ID mismatch j 600 t y ld r m i 10086843 10086842
ID mismatch zones 24099957 24099953
ID mismatch 6 million members 631691 631690
ID mismatch oikopleura 12851278 12851277
ID mismatch the happy ending 18651252 18651251
ID mismatch slert 15375152 15375151
ID mismatch ssrl 15775150 15775149
ID mismatch the longhorn 19575131 19575130
ID mismatch the fly 18314265 18314260
ID mismatch javier bardem 10152661 10152660
ID mismatch two women 23150500 23150498
ID mismatch vietnamese martyrs 23550497 23550496
ID mismatch turuk 23007722 23007721
ID mismatch non violent resistance 12605681 12605679
ID mismatch rohulky 14605667 14605666
ID mismatch executable 7496671 7496670
ID mismatch tattare 16205659 16205658
ID mismatch statistical control 15828178 15828177
ID mismatch 5 may 588714 588711
ID mismatch s vrit 15329550 15329549
ID mismatch the chucker 17329540 17329539
ID mismatch a 3

ID mismatch k o y l i 10550594 10550593
ID mismatch brief 5345085 5345083
ID mismatch the rms titanic 21114284 21114283
ID mismatch 10th edition 51267 51266
ID mismatch thoroughly modern millie 22666533 22666532
ID mismatch a liquid fueled engine 2078953 2078952
ID mismatch a teaching method 4392856 4392855
ID mismatch all sport 2192865 2192864
ID mismatch dr wolfensberger s teaching 6879508 6879507
ID mismatch burning pain 5442776 5442775
ID mismatch water related activities 23689045 23689044
ID mismatch pca 13527494 13527493
ID mismatch relative terms 14410604 14410602
ID mismatch thailand s royal family 16304963 16304962
ID mismatch the 20th century history 16504962 16504961
ID mismatch the author s message 16904960 16904957
ID mismatch the fourth philosophy 18388344 18388343
ID mismatch maglev 11165863 11165862
ID mismatch peperduur 13565851 13565850
ID mismatch quinton 14188365 14188364
ID mismatch energy detection 7253330 7253329
ID mismatch log 24128461 10988373
ID mismatch new 

KeyboardInterrupt: 

In [5]:
import numpy as np
true = "4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 4 4 4 3 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 2 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 3 4 3 4 4 4 4 0 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 0 0 4 4 4 4 0 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 3 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 0 4 3 4 4 4 4 4 0 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 0 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 0 4 4 4 0 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 2 0 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 0 4 4 4 0 3 4 4 4 4 4 4 4 4 3 3 4 4 4 4 4 4 4 3 4 4 4 2 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 2 4 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 0 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 0 4 4 4 2 4 4 3 4 4 4 4 4 3 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 3 4 4 4 4 0 4 4 4 4 4 4 0 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 2 4 4 4 4 3 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 3 4 4 0 4 4 4 0 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 2 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 3 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 3 4 4 4 1 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 2 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 0 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 3 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 2 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 0 4 4 4 4 4 4 2 4 4 0 4 4 4 4 4 3 4 4 4 4 4 2 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 3 4 3 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 3 4 4 4 4 0 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 0 4 0 3 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 0 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 0 4 4 1 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 0 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 0 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 3 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 3 4 4 4 4 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 0 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 3 4 4 0 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 1 0 4 4 4 4 4 4 0 4 4 4 4 4 4 0 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 3 4 0 4 4 4 4 4 4 4 4 0 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 0 4 4 0 4 4 4 4 3 4 2 0 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 2 2 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 2 3 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 3 4 4 4 4 4 4 4 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 3 4 2 4 4 4 4 4 4 4 1 4 4 4 4 4 4"
pred = "4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 0 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4"
true = np.array([int(s) for s in true.split()])
pred = np.array([int(s) for s in pred.split()])

In [10]:
dataset = [l.split("\t") for l in open("../files/dataset/dataset.tsv").read().split("\n")]
dataset_t = ["\t".join(l) for l in dataset if l[-1]!="none"]
dataset_f = ["\t".join(l) for l in dataset if l[-1]=="none"]
open("../files/dataset/dataset_t.tsv","w+").write("\n".join(dataset_t))
open("../files/dataset/dataset_f.tsv","w+").write("\n".join(dataset_f))

2994584